In [44]:
zuijin_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/最近的站点.csv'

In [45]:
zuijin = pd.read_csv(open(zuijin_name,encoding = 'gbk'))

In [46]:
zuijin.columns

Index(['小区CGI(*)', 'RRU经度', 'RRU纬度', '方位角'], dtype='object')

In [47]:
zuijin_shan = mt.add_sectors(zuijin, coords=['RRU经度', 'RRU纬度', '', '方位角'], has_z=False, sec_col='geometry', shape_dict={'radius': 2200, 'beam': 120, 'per_degree': 10})

In [48]:
xxx_p = nt.add_points(data_zhan_x , coords=[lon_x, lat_x,'',''])

In [50]:
res = gpd.sjoin(zuijin_shan,xxx_p)

In [53]:
res.to_clipboard()

# 计算最近的站点距离-19年10月

In [17]:
import pandas as pd
import geopandas as gpd
import numpy as np
import app.mydef as md
mt=md.MyTools()
con = mt.con()
import app.netools as ant
nt = ant.NetTools()

In [18]:
#导入的目录
x_enter_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/问题点.csv'
y_enter_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/现网lte小区.csv'
#导出的目录
data_out_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/结果.csv'

In [19]:
data_zhan_x = pd.read_csv(open(x_enter_name,encoding = 'gbk'))
data_zhan_y = pd.read_csv(open(y_enter_name,encoding = 'gbk'))

In [20]:
data_zhan_x.columns,data_zhan_y.columns

(Index(['问题点或问题小区cgi', '问题点中心经度', '问题点中心纬度'], dtype='object'),
 Index(['小区CGI(*)', '小区中文名', 'RRU经度', 'RRU纬度', '方位角'], dtype='object'))

In [21]:
# data_zhan_x = data_zhan_x[['问题点中心经度', 'grid', 'center']]
# data_zhan_x.columns = ['grid_center_lon_lat', 'lon_x', 'lat_x']

In [22]:
#选择关键列---必填-变量
id_x = '问题点或问题小区cgi'
lon_x = '问题点中心经度'
lat_x = '问题点中心纬度'
fanwei = 5000 #单位米，范围

id_y =  '小区CGI(*)'
lon_y = 'RRU经度'
lat_y = 'RRU纬度'
pi = 500 #循环过程中每次处理数据量

In [23]:
#制作图层
data_zhan_buff = nt.add_buffer(data_zhan_x , coords=[lon_x, lat_x,'',''],buff_m=fanwei)
data_zhan_pot = nt.add_points(data_zhan_y , coords=[lon_y, lat_y,'',''])

In [32]:
#数据处理-寻找最近的站点
nu = data_zhan_buff.shape
xhcs=nu[0]//pi
ys = nu[0]%pi
xh_cishu=list(range(xhcs))
data_append = pd.DataFrame()
#循环进行-整数部分处理
for i in xh_cishu:
    #批量sjoin
    x = i*pi
    y = x +pi
    data_zhan_t = data_zhan_buff.iloc[x:y,]
    data_jg_t = gpd.sjoin(data_zhan_t,data_zhan_pot)
    
    #去掉poi
    ss = data_jg_t.filter(regex="[^geometry,^lon_n,^lat_n]")
    #处理成结果格式
    lon1, lat1, lon2, lat2 = map(np.radians, 
                             [ss[lon_x], ss[lat_x], ss[lon_y], ss[lat_y]])
    a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
    ss['距离'] = 2*np.arcsin(np.sqrt(a))*6371*1000
    ss1= ss.loc[ss['距离']>-1]
    ss2=ss1.sort_values(by = '距离' ,ascending=True)
    ss3 = ss2.drop_duplicates(id_x)
    data_append = data_append.append(ss3)

    print('数据总计行列数',nu,'目前执行到',x,y,'本次圈到：',data_jg_t.shape,'有效',ss3.shape,'结果累计量：',data_append.shape)

数据总计行列数 (609, 4) 目前执行到 0 500 本次圈到： (1189495, 10) 有效 (500, 10) 结果累计量： (500, 10)


In [33]:
#余数部分进行
data_zhan_t=data_zhan_buff.iloc[(xh_cishu[-1]+1)*pi:(xh_cishu[-1]+1)*pi+ys,]
data_jg_t = gpd.sjoin(data_zhan_t,data_zhan_pot)

#去掉poi
ss = data_jg_t.filter(regex="[^geometry,^lon_n,^lat_n]")
#处理成结果格式
lon1, lat1, lon2, lat2 = map(np.radians, 
                         [ss[lon_x], ss[lat_x], ss[lon_y], ss[lat_y]])
a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
ss['距离'] = 2*np.arcsin(np.sqrt(a))*6371*1000
ss1= ss.loc[ss['距离']>-1]
ss2=ss1.sort_values(by = '距离' ,ascending=True)
ss3 = ss2.drop_duplicates(id_x)
data_append = data_append.append(ss3)
print((xh_cishu[-1]+1)*pi,(xh_cishu[-1]+1)*pi+ys,'完成','总计：',data_append.shape)

500 609 完成 总计： (609, 10)


In [31]:
ss3.shape

(109, 10)

In [26]:
#将结果更新到原表新增的列中
# data_append_re = data_append.reset_index()
data_zhan_x_res = data_zhan_x.reset_index()

In [27]:
data_res_ok = pd.merge(data_zhan_x_res,data_append,how='left',on=id_x)

In [28]:
data_res_ok.sort_values('index',inplace = True)
data_res_ok_drop = data_res_ok.drop_duplicates('index')

In [29]:
#导出
data_res_ok_drop.to_csv(data_out_name ,encoding = 'gbk',index=False)

In [30]:
data_zhan_buff.shape,data_res_ok_drop.shape

((609, 4), (609, 13))

# 结束

In [1]:
from __dis import *
b=get_distance(114.225436, 31.552066, 114.245436,  31.572066)
a=get_fw(114.225436, 31.552066, 114.225436, 31.552066)
print(a,b)

0.0 2921.723252391541


In [ ]:
import pandas as pd
from __dis import *
data_a_y = pd.read_excel('A.xlsx',sheet_name = 'Sheet1')
data_b_y = pd.read_excel('A.xlsx',sheet_name = 'Sheet2')
data_a_y.iloc[1:3,:3]

In [ ]:
data_b_y.iloc[:1]

In [ ]:
#------------------------计算最近距离小区----------
data_a = data_a_y.copy()
data_b = data_b_y.copy()
out_cgi_bok=[]
out_lon_bok=[]
out_lat_bok=[]
out_juli=[]
for x in range(len(data_a.index)):
    cgi_a,lon_a,lat_a=data_a.iloc[x,:3]
    juli = 10000000
    for x in range(len(data_b.index)):
        cgi_b,lon_b,lat_b=data_b.iloc[x,:]
        julia=get_distance(lon_a,lat_a,lon_b,lat_b)
#         if julia < juli and julia>50:   #不计算50米内的小区
        if julia < juli:    
            juli = julia
            cgi_bok = cgi_b
            lon_bok = lon_b
            lat_bok = lat_b
    out_cgi_bok.append(cgi_bok)
    out_lon_bok.append(lon_bok)
    out_lat_bok.append(lat_bok)
    out_juli.append(juli)
data_a['cig_b'] = out_cgi_bok
data_a['lon_b'] = out_lon_bok
data_a['lat_b'] = out_lat_bok
data_a['juli_b'] = out_juli
data_a.to_excel('A-最近距离.xlsx',sheet_name = '最近距离',index = False)

In [ ]:
#------------------------计算最远距离小区-------
data_a = data_a_y.copy()
data_b = data_b_y.copy()
out_cgi_bok=[]
out_lon_bok=[]
out_lat_bok=[]
out_juli=[]
for x in range(len(data_a.index)):
    cgi_a,lon_a,lat_a=data_a.iloc[x,:]
    juli = 0
    for x in range(len(data_b.index)):
        cgi_b,lon_b,lat_b=data_b.iloc[x,:]
        julia=get_distance(lon_a,lat_a,lon_b,lat_b)
        if julia > juli:
            juli = julia
            cgi_bok = cgi_b
            lon_bok = lon_b
            lat_bok = lat_b
    out_cgi_bok.append(cgi_bok)
    out_lon_bok.append(lon_bok)
    out_lat_bok.append(lat_bok)
    out_juli.append(juli)
data_a['cig_b'] = out_cgi_bok
data_a['lon_b'] = out_lon_bok
data_a['lat_b'] = out_lat_bok
data_a['juli_b'] = out_juli
data_a.to_excel('最远距离.xlsx',index = False)

# 计算最近的小区&朝向点位置

In [17]:
import pandas as pd
import geopandas as gpd
import numpy as np
import app.mydef as md
mt=md.MyTools()
con = mt.con()
import app.netools as ant
nt = ant.NetTools()

In [18]:
#导入的目录
x_enter_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/问题点.csv'
y_enter_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/现网lte小区.csv'
#导出的目录
data_out_name = 'g:/1-规划/1其他工作输出/20191205更新任务清单-规划转优化给cig/结果.csv'

In [19]:
data_zhan_x = pd.read_csv(open(x_enter_name,encoding = 'gbk'))
data_zhan_y = pd.read_csv(open(y_enter_name,encoding = 'gbk'))

In [20]:
data_zhan_x.columns,data_zhan_y.columns

(Index(['问题点或问题小区cgi', '问题点中心经度', '问题点中心纬度'], dtype='object'),
 Index(['小区CGI(*)', '小区中文名', 'RRU经度', 'RRU纬度', '方位角'], dtype='object'))

In [21]:
# data_zhan_x = data_zhan_x[['问题点中心经度', 'grid', 'center']]
# data_zhan_x.columns = ['grid_center_lon_lat', 'lon_x', 'lat_x']

In [22]:
#选择关键列---必填-变量
id_x = '问题点或问题小区cgi'
lon_x = '问题点中心经度'
lat_x = '问题点中心纬度'
fanwei = 5000 #单位米，范围

id_y =  '小区CGI(*)'
lon_y = 'RRU经度'
lat_y = 'RRU纬度'
pi = 500 #循环过程中每次处理数据量

In [23]:
#制作图层
data_zhan_buff = nt.add_buffer(data_zhan_x , coords=[lon_x, lat_x,'',''],buff_m=fanwei)
data_zhan_pot = nt.add_points(data_zhan_y , coords=[lon_y, lat_y,'',''])

In [32]:
#数据处理-寻找最近的站点
nu = data_zhan_buff.shape
xhcs=nu[0]//pi
ys = nu[0]%pi
xh_cishu=list(range(xhcs))
data_append = pd.DataFrame()
#循环进行-整数部分处理
for i in xh_cishu:
    #批量sjoin
    x = i*pi
    y = x +pi
    data_zhan_t = data_zhan_buff.iloc[x:y,]
    data_jg_t = gpd.sjoin(data_zhan_t,data_zhan_pot)
    
    #去掉poi
    ss = data_jg_t.filter(regex="[^geometry,^lon_n,^lat_n]")
    #处理成结果格式
    lon1, lat1, lon2, lat2 = map(np.radians, 
                             [ss[lon_x], ss[lat_x], ss[lon_y], ss[lat_y]])
    a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
    ss['距离'] = 2*np.arcsin(np.sqrt(a))*6371*1000
    ss1= ss.loc[ss['距离']>-1]
    ss2=ss1.sort_values(by = '距离' ,ascending=True)
    
    ss3 = ss2.drop_duplicates(id_x)
    data_append = data_append.append(ss3)

    print('数据总计行列数',nu,'目前执行到',x,y,'本次圈到：',data_jg_t.shape,'有效',ss3.shape,'结果累计量：',data_append.shape)

数据总计行列数 (609, 4) 目前执行到 0 500 本次圈到： (1189495, 10) 有效 (500, 10) 结果累计量： (500, 10)


In [38]:
ss2.columns

Index(['问题点或问题小区cgi', '问题点中心经度', '问题点中心纬度', 'index_right', '小区CGI(*)', '小区中文名',
       'RRU经度', 'RRU纬度', '方位角', '距离'],
      dtype='object')

In [37]:
groupby_x = ss2.groupby(by = id_x)

In [ ]:
for name ,data_t in groupby_x:
    data_t

In [33]:
#余数部分进行
data_zhan_t=data_zhan_buff.iloc[(xh_cishu[-1]+1)*pi:(xh_cishu[-1]+1)*pi+ys,]
data_jg_t = gpd.sjoin(data_zhan_t,data_zhan_pot)

#去掉poi
ss = data_jg_t.filter(regex="[^geometry,^lon_n,^lat_n]")
#处理成结果格式
lon1, lat1, lon2, lat2 = map(np.radians, 
                         [ss[lon_x], ss[lat_x], ss[lon_y], ss[lat_y]])
a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
ss['距离'] = 2*np.arcsin(np.sqrt(a))*6371*1000
ss1= ss.loc[ss['距离']>-1]
ss2=ss1.sort_values(by = '距离' ,ascending=True)
ss3 = ss2.drop_duplicates(id_x)
data_append = data_append.append(ss3)
print((xh_cishu[-1]+1)*pi,(xh_cishu[-1]+1)*pi+ys,'完成','总计：',data_append.shape)

500 609 完成 总计： (609, 10)


In [31]:
ss3.shape

(109, 10)

In [26]:
#将结果更新到原表新增的列中
# data_append_re = data_append.reset_index()
data_zhan_x_res = data_zhan_x.reset_index()

In [27]:
data_res_ok = pd.merge(data_zhan_x_res,data_append,how='left',on=id_x)

In [28]:
data_res_ok.sort_values('index',inplace = True)
data_res_ok_drop = data_res_ok.drop_duplicates('index')

In [29]:
#导出
data_res_ok_drop.to_csv(data_out_name ,encoding = 'gbk',index=False)

In [30]:
data_zhan_buff.shape,data_res_ok_drop.shape

((609, 4), (609, 13))

# 结束